<a href="https://colab.research.google.com/github/Krushna-sahoo/Deep-fake-detection/blob/main/extraction_and_combination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install opencv-python-headless facenet-pytorch numpy


In [3]:
import cv2
import os
import numpy as np
from facenet_pytorch import MTCNN

#  Define paths
VIDEO_FOLDER = "/content/drive/MyDrive/Deep-fake-detection"  # Folder with raw videos
OUTPUT_FOLDER = "/content/drive/MyDrive/Deep-fake-detection/processed_faces"  # Folder for face-only videos
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

#  Initialize MTCNN
mtcnn = MTCNN(keep_all=False, select_largest=True)

def process_video(video_path, output_path, frame_limit=100):
    """Extract face from each frame and create a face-only video."""
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    face_frames = []

    while cap.isOpened() and frame_count < frame_limit:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB for MTCNN
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect face and get bounding box
        boxes, _ = mtcnn.detect(rgb_frame)

        if boxes is not None and len(boxes) > 0:
            x1, y1, x2, y2 = map(int, boxes[0])  # Get first detected face

            # Ensure bounding box is within image bounds
            h, w, _ = frame.shape
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)

            # Extract face from original BGR image
            face = frame[y1:y2, x1:x2]

            if face.size > 0:
                # Resize for consistency
                face_resized = cv2.resize(face, (160, 160), interpolation=cv2.INTER_LANCZOS4)

                # Store extracted face frame
                face_frames.append(face_resized)

        frame_count += 1

    cap.release()

    # Convert extracted faces into a new video
    if len(face_frames) > 0:
        height, width, _ = face_frames[0].shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = 10  # Adjust FPS as needed
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        for face in face_frames:
            video_writer.write(face)

        video_writer.release()
        print(f"Face-only video saved: {output_path}")
    else:
        print(f"No faces detected in {video_path}")

def process_all_videos(video_folder, output_folder, frame_limit=100):
    """Process all videos in a folder and generate face-only videos."""
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

    if not video_files:
        print(" No videos found in the folder!")
        return

    for video_file in video_files:
        video_path = os.path.join(video_folder, video_file)
        output_path = os.path.join(output_folder, f"processed_{video_file}")

        print(f" Processing: {video_file}")
        process_video(video_path, output_path, frame_limit)

#  Process all videos in the folder
process_all_videos(VIDEO_FOLDER, OUTPUT_FOLDER)

print(" Face-only dataset creation complete!")


 Processing: id9_id16_0000.mp4
Face-only video saved: /content/drive/MyDrive/Deep-fake-detection/processed_faces/processed_id9_id16_0000.mp4
 Processing: id9_id16_0002.mp4
Face-only video saved: /content/drive/MyDrive/Deep-fake-detection/processed_faces/processed_id9_id16_0002.mp4
 Processing: id9_id16_0008.mp4
Face-only video saved: /content/drive/MyDrive/Deep-fake-detection/processed_faces/processed_id9_id16_0008.mp4
 Processing: id9_id16_0001.mp4
Face-only video saved: /content/drive/MyDrive/Deep-fake-detection/processed_faces/processed_id9_id16_0001.mp4
 Processing: id9_id16_0003.mp4
Face-only video saved: /content/drive/MyDrive/Deep-fake-detection/processed_faces/processed_id9_id16_0003.mp4
 Processing: id9_id16_0005.mp4
Face-only video saved: /content/drive/MyDrive/Deep-fake-detection/processed_faces/processed_id9_id16_0005.mp4
 Processing: id9_id16_0007.mp4
Face-only video saved: /content/drive/MyDrive/Deep-fake-detection/processed_faces/processed_id9_id16_0007.mp4
 Processing: 